In [1]:
import warnings

warnings.filterwarnings("ignore")

from src import utils
from src.utils import VotingSchemas
from src.outcomes import *
from src.happiness_level import HappinessLevel
from src.strategic_voting_risk import StrategicVoting
import os
import numpy as np

In [2]:
path_voting = "input/voting_result.json"
if "env" in os.environ:
    if os.environ["env"] == "docker":
        path_voting = "/input/voting_result.json"

In [3]:
voting = utils.read_voting(path_voting, table_name="voting5")
voting.to_pandas()

,voter_0,voter_1,voter_2,voter_3,voter_4,voter_5
preference_0,C,B,C,C,B,A
preference_1,A,D,D,D,A,D
preference_2,D,C,A,B,D,B
preference_3,B,A,B,A,C,C


## Strategic Voting

In [4]:
result = plurality_outcome(voting)
result

{'A': 1, 'B': 2, 'C': 3, 'D': 0}

Note that **voter 5** is the only voter that can engage in strategic voting. Voter 5 prefers $B$ over the winner $C$. Hence, they can increase their $\text{Happiness Level}$ by placing $B$ as their first preference, resulting in a tie between $C$ and $B$ in the voting outcome (for plurality, voting for two, Borda). Since ties are resolved in alphabetical order, $B$ wins over $C$.

The table below displays the two voting strategies available to voter 5.

In [5]:
happiness_level = HappinessLevel(voting, result.winner, VotingSchemas.PLURALITY_VOTING).run()
strategic_voting = StrategicVoting(voting, happiness_level, plurality_outcome).run(show=True)

Strategic Voting


,voter,strategic_voting,new_result,strategic_H,previous_H,strategic_overall_H,previous_overall_H
0,5,"[B, A, D, C]",B,0.410487,0.0,2.820973,3.410487
1,5,"[B, D, A, C]",B,0.410487,0.0,2.820973,3.410487


Best Strategic Voting


,voter,strategic_voting,new_result,strategic_H,previous_H,strategic_overall_H,previous_overall_H
0,5,"[B, A, D, C]",B,0.410487,0.0,2.820973,3.410487


Risk: 0.13682886358890622


The table displays the best strategic voting for each voter

In [6]:
strategic_voting.best

,voter,strategic_voting,new_result,strategic_H,previous_H,strategic_overall_H,previous_overall_H
0,5,"[B, A, D, C]",B,0.410487,0.0,2.820973,3.410487


Strategic Voting Risk

In [7]:
strategic_voting.risk

0.13682886358890622

In [8]:
import numpy as np
from typing import Callable
from itertools import permutations
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from src.optimized_code import read_voting_, compute_risk, plurality_outcome_op, veto_outcome_op, borda_outcome_op, for_two_outcome_op, compute_risk_parallel, compute_risk_combinations, happiness_level_total
import os
import json
import multiprocessing

ImportError: cannot import name 'compute_risk_parallel' from 'src.optimized_code' (/Users/macbook/Desktop/mas-group-1/src/optimized_code.py)

In [ ]:
def compute_voter_risk(preferences: np.ndarray, result: int, initial_happinesses: np.ndarray, i: int, schema_outcome_f: Callable) -> float:
    initial_happiness = initial_happinesses[i]
    vwr_idx = np.flatnonzero(preferences[:, i] == result)
    if vwr_idx.size == 0:
        return 0 

    vwr = vwr_idx[0]
    best_happiness = initial_happiness

    perms = list(permutations(preferences[:, i]))
    perm_array = np.array(perms)

    new_voting = preferences.copy()
    for perm in perm_array:
        idx = np.argwhere(perm == result)[0][0]
        if idx >= vwr:
            new_voting[:, i] = perm
            new_result = schema_outcome_f(new_voting)
            new_happiness = happiness_level(preferences, i, new_result)
            best_happiness = max(best_happiness, new_happiness)

    return best_happiness - initial_happiness

def compute_risk(preferences: np.ndarray, schema_outcome_f: Callable) -> float:
    result = schema_outcome_f(preferences)
    initial_happinesses = happiness_level_total(preferences, result)
    num_unhappy_voters = np.count_nonzero(initial_happinesses != 1)

    total_risk = sum(compute_voter_risk(preferences, result, initial_happinesses, voter, schema_outcome_f) 
                    for voter in range(preferences.shape[1]))

    if num_unhappy_voters == 0:
        return 0
    return total_risk / num_unhappy_voters

def compute_risk_parallel(preferences: np.ndarray, schema_outcome_f: Callable) -> float:
    result = schema_outcome_f(preferences)
    initial_happinesses = happiness_level_total(preferences, result)
    num_unhappy_voters = np.count_nonzero(initial_happinesses != 1)


    risk = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(compute_voter_risk)(preferences,result,initial_happinesses,i,schema_outcome_f) for i in range(preferences.shape[1]))
    total_risk = risk.sum()

    if num_unhappy_voters == 0:
        return 0
    return total_risk / num_unhappy_voters